In [1]:
from __future__ import print_function
import sys
import json
import os
import subprocess
import re
import string
from bs4 import BeautifulSoup
import urllib2
import csv
import copy

In [2]:
collection = "gov2"

projectDir = "/home/fj9124/projects/ir/seq_kb_ir/"
optParamsDir = os.path.join(projectDir, "optParams")
annotateProg = os.path.join(optParamsDir, "annotate.py")
print("annotateProg =", annotateProg)

configsDir = os.path.join(projectDir, "configs")
colConfigsDir = os.path.join(configsDir, collection)
if collection == "gov2":
    colInConfigsFile = os.path.join(colConfigsDir, "06.topics.701-850.txt")
print ("colInConfigsFile =", colInConfigsFile)
colOutConfigsFile = os.path.join(colConfigsDir, "indriRunQuery.cfg")
print ("colOutConfigsFile =", colOutConfigsFile)

indexDir = "/scratch/index/indri_5_7/"
colIndexDir = os.path.join(indexDir, collection) 
print("collection index Dir =", colIndexDir)

colRunsFile = os.path.join(projectDir, "runs", collection, "indriRunQuery.run")
print("colRunsFile =", colRunsFile)

if collection == "gov2":
    colQrelsFile = os.path.join(projectDir, "qrels", collection, "qrels.tb06.top50")
print("colQrelsFile =", colQrelsFile)

colEvalsFile = os.path.join(projectDir, "evals", collection, "indriRunQuery.evals")
print("colEvalsFile =", colEvalsFile)

stopwordsFile = "/backup/ir/conf/stopwords.txt"

annotateProg = /home/fj9124/projects/ir/seq_kb_ir/optParams/annotate.py
colInConfigsFile = /home/fj9124/projects/ir/seq_kb_ir/configs/gov2/06.topics.701-850.txt
colOutConfigsFile = /home/fj9124/projects/ir/seq_kb_ir/configs/gov2/indriRunQuery.cfg
collection index Dir = /scratch/index/indri_5_7/gov2
colRunsFile = /home/fj9124/projects/ir/seq_kb_ir/runs/gov2/indriRunQuery.run
colQrelsFile = /home/fj9124/projects/ir/seq_kb_ir/qrels/gov2/qrels.tb06.top50
colEvalsFile = /home/fj9124/projects/ir/seq_kb_ir/evals/gov2/indriRunQuery.evals


In [3]:
with open(stopwordsFile, 'r') as f:
    stopwords = list(csv.reader(f, delimiter='\n'))
    stopwords = {s[0] for s in stopwords}
    print(stopwords)

set(['lest', 'all', 'just', 'less', 'being', 'indeed', 'over', 'anyway', 'hath', 'own', 'through', 'wherewith', 'hereto', 'whereinto', 'where', 'still', 'thyself', 'its', 'before', 'one', 'whereabouts', 'whereon', 'yourselves', 'halves', 'selves', 'herself', 'whoever', 'had', 'behind', 'hereabouts', 'should', 'meantime', 'to', 'must', 'choose', 'whatsoever', 'under', 'inwards', 'of', 'has', 'might', 'thereafter', 'latterly', 'outside', 'do', 'hindmost', 'his', 'somebody', 'around', 'thereby', 'get', 'very', 'thee', 'she', 'none', 'underneath', 'whichever', 'cannot', 'every', 'whatever', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'like', 'well', 'several', 'hereafter', 'whilst', 'did', 'ever', 'seldom', 'whither', 'via', 'this', 'someone', 'either', 'though', 'always', 'become', 'thereupon', 'sometime', 'whomever', 'upward', 'therein', 'canst', 'because', 'often', 'our', 'doing', 'howsoever', 'yourself', 'some', 'somehow', 'up', 'namely', 'towards', 'are', 'notwithstanding'

In [4]:
def conceptNetText(expr):
    if expr == "":
        return("")
    expr = expr.replace(' ', '_')
    textUrl = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + expr
    #print("textUrl =", textUrl)
    html_doc = urllib2.urlopen(textUrl)
    soup = BeautifulSoup(html_doc, "lxml")
    jsonContent = soup.find('body').find('p').contents[0]
    jsonContent = json.loads(jsonContent)
    conceptText = jsonContent[u'uri']
    
    conceptTextB = str(os.path.basename(conceptText))#.replace('_', ' ')
    conceptTextD = str(os.path.dirname(conceptText))
    if conceptTextD == "/c/en":
    #if numFound != 0:
        return(conceptTextB)
    else:
        return("")

In [5]:
def conceptNetLookup(conceptText):
    lookupUrl = "http://conceptnet5.media.mit.edu/data/5.4" + conceptText
    #print("lookupUrl =", lookupUrl)
    html_doc = urllib2.urlopen(lookupUrl)
    soup = BeautifulSoup(html_doc, "lxml")
    #print(soup.prettify())
    jsonContent = soup.find('body').find('p').contents[0]
    jsonContent = json.loads(jsonContent)
    numFound = int(jsonContent[u'numFound'])
    if numFound != 0:
        return(conceptText)
    else:
        return("")

In [6]:
def genBigrams(unigrams):
    a = copy.deepcopy(unigrams)
    b = copy.deepcopy(unigrams)
    del a[0]
    del b[-1]
    s = [b[i] + " " + j for i, j in enumerate(a) if j not in stopwords and b[i] not in stopwords ]
    return(s)

In [7]:
def entityRecognition(text):
    # check if a derivation of expression exists in conceptnet and what is it? 
    concepts = []
    #origText = "Relevant documents will include those on historical exploration and drilling as well as history of regulatory bodies. Relevant are history of the oil industry in various states, even if drilling began in 1950 or later"
    #unigrams = origText.split()
    unigrams = text.split()
    bigrams = genBigrams(unigrams)
    #print ("bigrams =", bigrams)
    unigramConcepts = set()
    for unigram in unigrams:
        if unigram in stopwords:
            continue
        #print("unigram =", unigram)
        uri = "/c/en/" + conceptNetText(conceptNetText(unigram))
        unigramConcept = conceptNetLookup(uri).replace("/c/en/", "")
        #print ("unigramConcept =", unigramConcept)
        if unigramConcept != "":
            unigramConcepts.add(unigramConcept)
    #print("unigramConcepts =", unigramConcepts)
    bigramConcepts = set()
    for bigram in bigrams:
        bigram = bigram.replace(' ', '_')
        uri = "/c/en/" + conceptNetText(conceptNetText(bigram))
        bigramConcept = conceptNetLookup(uri).replace("/c/en/", "")
        #print("bigramConcept =", bigramConcept)
        if bigramConcept != "":
            bigramConcepts.add(bigramConcept)
    #print("bigramConcepts =", bigramConcepts)
    return(unigramConcepts, bigramConcepts)

In [8]:
# testing entityRecognition
text = "Relevant documents will include those on historical exploration and drilling as well as history of regulatory bodies. Relevant are history of the oil industry in various states, even if drilling began in 1950 or later"
unigramConcepts, bigramConcepts = entityRecognition(text)
print(unigramConcepts, bigramConcepts)

set(['body', 'late', 'begin', 'relevant', 'drill', 'industry', 'will', 'state', 'oil', 'exploration', '1950', 'historical', 'regulatory', 'various', 'include', 'document', 'history']) set(['oil_industry', 'relevant_document', 'regulatory_body'])


In [9]:
#example: origText = "Relevant documents will include those on historical exploration and drilling as well as history of regulatory bodies. Relevant are history of the oil industry in various states, even if drilling began in 1950 or later"
def annotateTextAlchemyApi(origText):
    #print("origText =", origText)
    cmd = ' '.join([annotateProg, "\"" + origText + "\""])
    #print("cmd =", cmd)
    annotateResults = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
    #print("annotateResults =", annotateResults)
    #annotateResults = subprocess.check_output([annotateProg, "\"" + origText + "\""])
    print("annotateResults =", annotateResults[0:500], "...")
    annotateResultsJson = re.split('## Response Object ##|## Keywords ##',annotateResults)[1].strip()
    annotateResultsJsonParsed = json.loads(annotateResultsJson)
    freebaseEntities = [ j[u'freebase'] for j in annotateResultsJsonParsed["concepts"] if u'freebase' in j]
    #print (freebaseEntities)
    entityTexts = [ str(j['text'].encode('utf8')) for j in annotateResultsJsonParsed["concepts"]]
    #print (entityTexts)
    return(entityTexts)

In [10]:
# testing annotation:
origText = "Relevant documents will include those on historical exploration and drilling as well as history of regulatory bodies. Relevant are history of the oil industry in various states, even if drilling began in 1950 or later"
annotateTextAlchemyApi(origText)

annotateResults = text = Relevant documents will include those on historical exploration and drilling as well as history of regulatory bodies. Relevant are history of the oil industry in various states, even if drilling began in 1950 or later
## Response Object ##
{
    "status": "OK", 
    "totalTransactions": "4", 
    "language": "english", 
    "entities": [
        {
            "relevance": "0.920237", 
            "count": "1", 
            "type": "FieldTerminology", 
            "text": "oil industry"
   ...


['Petroleum', 'Renaissance', 'History', 'Petroleum industry']

In [11]:
def getWikipediaEn(freebaseEntityUrl):
    #print("extracting keys from /wikipedia/en namespace of freespace")
    #print ("freebase url =", freebaseEntities[0])
    #html_doc = urllib2.urlopen(freebaseEntities[0])
    html_doc = urllib2.urlopen(freebaseEntityUrl)
    soup = BeautifulSoup(html_doc, "lxml")

    freebaseContent = soup.text.split('\n')

    #  "the key [that] can be used to formulate a URL to the corresponding English article on Wikipedia"
    keyWikipediaEn = "key:wikipedia.en "

    wikipediaEn = [re.sub(keyWikipediaEn, '', str(i)).strip() for i in freebaseContent if keyWikipediaEn in i]
    wikipediaEn = [re.sub('[^a-zA-Z]+', ' ', i) for i in wikipediaEn]
    wikipediaEn = [re.sub( '\s+', ' ', i ).strip() for i in wikipediaEn]
    #print("wikipediaEn =", wikipediaEn)
    return(wikipediaEn)

#keyWikipediaObjectKeyEn = "ns:type.object.key    ns:wikipedia.en."

#wikipediaObjectKeyEn = [re.sub(keyWikipediaObjectKeyEn, '', str(i)).strip() for i in freebaseContent if keyWikipediaObjectKeyEn in i]
#wikipediaObjectKeyEn = [re.sub('[^a-zA-Z]+', ' ', i) for i in wikipediaObjectKeyEn]
#wikipediaObjectKeyEn = [re.sub( '\s+', ' ', i ).strip() for i in wikipediaObjectKeyEn]
#print("wikipediaObjectKeyEn =", wikipediaObjectKeyEn)

#print(set(wikipediaObjectKeyEn) - set(wikipediaEn))



In [12]:
print("parse queries file")
regex = re.compile('[%s]' % re.escape(string.punctuation))
queries = dict()
with open(colInConfigsFile, 'r') as infile:
    xmlDoc = infile.read()
    soup = BeautifulSoup(xmlDoc, "lxml")
    topTags = soup.findAll('top')
    #print(soup.prettify())
    for topTag in topTags:
        numTag = topTag.findAll('num')[0]
        qNo = int(str(numTag.contents[0]).replace("Number: ", ""))
        if qNo < 801:
            continue
        #narrTag = numTag.findAll('title')[0].findAll('desc')[0].findAll('narr')[0]
        #qText = str(numTag.findAll('title')[0].contents[0])
        qText = str(numTag.findAll('title')[0].findAll('desc')[0].contents[0])
        #print(qTextTag)
        qText = qText.replace("Description:","").replace("\n", " ").strip()
        #narr = narrTag.string.replace("Narrative:","").replace("\n", " ").strip()
        qText = regex.sub(' ', qText)
        qText = re.sub( '\s+', ' ', qText)
        queries[qNo] = qText
print ("queries =", queries)

parse queries file
queries = {801: 'Describe the origin nature extent of spread and means of controlling kudzu ', 802: 'What is the impact of volcano eruptions on global temperature ', 803: 'Give the history of this holiday and the various ways of celebrating May Day ', 804: 'Describe resolutions proposed and legislation passed to ban the cloning of humans and the rationale for the bans ', 805: 'Describe the Identify Theft Passport issued to identity theft victims to show to creditors and law enforcement officers questioning their credit worthiness or innocence ', 806: 'What is Doctors Without Borders Medecins Sans Frontieres and what do they do ', 807: 'Describe the nature and history of sugar tariff rate quotas in the United States ', 808: 'What information is available on the involvement of the North Korean Government in counterfeiting of US currency ', 809: 'Identify wastewater treatment projects that involve constructed or natural wetlands ', 810: 'Provide information regarding ti

In [13]:
def mrfQueryGen(conceptText, odW, uwW):
    mrfText = "#weight( "
    mrfText += str(1 - odW - uwW) + " #combine( " + conceptText + " ) "
    conceptText_ = conceptText.split(" ")
    #if len(conceptText_) > 2:
    odText = ""
    uwText = ""
    for i in range(1, len(conceptText_)):
        odText += " #od4( " + ' '.join(conceptText_[i-1:i+1]) + " ) "
        uwText += " #uw17( " + ' '.join(conceptText_[i-1:i+1]) + " ) "
    if len(odText) > 10: # if it is not just tags
        mrfText += str(odW) + " #combine( " + odText + " ) "
        mrfText += str(uwW) + " #combine( " + uwText + " ) "
    mrfText += " ) "
    return(mrfText)

In [14]:
def generateQuery(dirCoeff, intCoeff, odW, uwW):
    soupNew = BeautifulSoup("", 'lxml')
    parameters_tag = soupNew.new_tag("parameters")
    index_tag = soupNew.new_tag("index")
    index_tag.string = colIndexDir
    parameters_tag.append(index_tag)
    for qNo, qText in queries.iteritems():
        doc_tag = soupNew.new_tag("query")

        docno_tag = soupNew.new_tag("number")
        docno_tag.string = str(qNo)
        doc_tag.append(docno_tag)
        
        text_tag = soupNew.new_tag("text")
        
        text_tag.string = "#weight( "
        text_tag.string += str(1- intCoeff) + " #combine( " + qText + ")"
        
        unigramConcepts, bigramConcepts = entityRecognition(qText)
        concepts = unigramConcepts | bigramConcepts
        #concepts = annotateTextAlchemyApi(qText)
        
        text_tag.string += str(intCoeff) + " #combine("
        for concept in concepts:
            #print("concept =", concept.decode('utf-8'))
            text_tag.string += mrfQueryGen(concept.decode('utf-8'), odW, uwW)
        text_tag.string += " )"

        text_tag.string += " )"
        
        doc_tag.append(text_tag)

        parameters_tag.append(doc_tag)
        #print(doc_tag)

    rule_tag = soupNew.new_tag("rule")
    rule_tag.string = "method:dir,mu:" + str(dirCoeff)
    #rule_tag.string = "method:two"
    parameters_tag.append(rule_tag)

    #intCoeff_tag = soupNew.new_tag("intCoeff")
    #intCoeff_tag.string = "0.8"
    #parameters_tag.append(intCoeff_tag)

    threads_tag = soupNew.new_tag("threads")
    threads_tag.string = "32"
    parameters_tag.append(threads_tag)

    count_tag = soupNew.new_tag("count")
    count_tag.string = "1000"
    parameters_tag.append(count_tag)

    trecFormat_tag = soupNew.new_tag("trecFormat")
    trecFormat_tag.string = "true"
    parameters_tag.append(trecFormat_tag)

    soupNew.append(parameters_tag)
    print(soupNew.prettify())
    #print("outFileName =", outFileName)

    with open(colOutConfigsFile, 'w') as outfile:
        outfile.write(str(soupNew))

In [15]:
dirrCoeff= 2600
intCoeff = 0.8
odW = 0.15
uwW = 0.15
generateQuery(dirrCoeff, intCoeff, odW, uwW)

concept = origin
concept = control
concept = kudzu
concept = nature
concept = describe
concept = spread
concept = extent
concept = mean
concept = impact
concept = volcano_eruption
concept = what
concept = global_temperature
concept = temperature
concept = volcano
concept = global
concept = eruption
concept = give
concept = may
concept = may_day
concept = various
concept = way
concept = holiday
concept = day
concept = celebrate
concept = history
concept = human
concept = propose
concept = clone
concept = describe
concept = legislation
concept = rationale
concept = pass
concept = ban
concept = resolution
concept = enforcement
concept = victim
concept = worthiness
concept = show
concept = describe
concept = identify
concept = theft
concept = innocence
concept = law
concept = law_enforcement
concept = identity
concept = creditor
concept = identity_theft
concept = credit_worthiness
concept = question
concept = credit
concept = officer
concept = passport
concept = issue
concept = what
concep

In [16]:
cmd = ' '.join(["IndriRunQuery", colOutConfigsFile, ">", colRunsFile])
print("cmd =", cmd)
subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()

cmd = IndriRunQuery /home/fj9124/projects/ir/seq_kb_ir/configs/gov2/indriRunQuery.cfg > /home/fj9124/projects/ir/seq_kb_ir/runs/gov2/indriRunQuery.run


''

In [17]:
cmd = ' '.join(["trec_eval", "-q", colQrelsFile, colRunsFile, ">", colEvalsFile])
print("cmd =", cmd)
subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()

cmd = trec_eval -q /home/fj9124/projects/ir/seq_kb_ir/qrels/gov2/qrels.tb06.top50 /home/fj9124/projects/ir/seq_kb_ir/runs/gov2/indriRunQuery.run > /home/fj9124/projects/ir/seq_kb_ir/evals/gov2/indriRunQuery.evals


''

In [18]:
with open(colEvalsFile, "r") as infile:
    reader = csv.reader(infile, delimiter='\t')
    precTable = list(reader)
    #print(precTable)
    precTable = {i[0].strip(): float(i[2].strip()) for i in precTable if i[1].strip()=="all" and i[0].strip() != "runid"}
    print(precTable['map'])

0.2426
